## Hamiltonian reduction

In [1]:
from quchem.Hamiltonian_Generator_Functions import *
from quchem.Graph import *
### HAMILTONIAN start
Molecule = 'LiH'
geometry = None # [('H', (0., 0., 0.)), ('H', (0., 0., 0.74))]
basis = 'sto-3g'


### Get Hamiltonian
Hamilt = Hamiltonian(Molecule,
                     run_scf=1, run_mp2=1, run_cisd=1, run_ccsd=1, run_fci=1,
                     basis=basis,
                     multiplicity=1,
                     geometry=geometry)  # normally None!
QubitHamiltonian = Hamilt.Get_Qubit_Hamiltonian(threshold=None, transformation='BK')
### HAMILTONIAN end


## Use NOON to remove terms for UCCSD operators


In [2]:
NOON_spins_combined, NMO_basis = Hamilt.Get_NOON()

NOON_spins_combined

array([1.99991509e+00, 1.96744892e+00, 2.71958861e-02, 7.94453898e-05,
       2.68032899e-03, 2.68032899e-03])

The LiH ground state under JW is:

$$|\psi \rangle_{HF}^{ground} = | 1 1 1 1 \:  0 0 0 0 0 0 0 0 \rangle$$

and under BK is:

$$|\psi \rangle_{HF}^{ground} = | 1 0 1 0 0 0 0 0 0 0 0 0 \rangle$$

Looking at the NOON we see that orbitals with indices:
- 0,1 remain OCCUPIED
- 6,7 remain UN_occupied


In [3]:
from quchem.Ansatz_Generator_Functions import *

ansatz_obj = BK_Qubit_Reduction(QubitHamiltonian,
                             Hamilt.molecule.n_electrons,
                             Hamilt.molecule.n_qubits)


#### Use this to reduce Hamiltonian:

In [4]:
list_of_qubit_indices_to_remove = [0,1,6,7]
reduced_Qubit_Hamiltonian = ansatz_obj.Remove_indices_from_Hamiltonian(list_of_qubit_indices_to_remove)
print('Hamiltonian size reduced: {} --> {}'.format(len(list(QubitHamiltonian)), len(list(reduced_Qubit_Hamiltonian))))

Hamiltonian size reduced: 631 --> 450


#### Re_label Hamiltonian:

In [5]:
qubit_re_label_dict, reduced_RE_LABELLED_Qubit_Hamiltonian = ansatz_obj.Re_label_Hamiltonian(reduced_Qubit_Hamiltonian)
reduced_RE_LABELLED_Qubit_Hamiltonian

(-6.030955269134001+0j) [] +
(-0.0581183863373224+0j) [X0] +
(0.00970296459034371+0j) [X0 X1 X2 X3] +
(-0.0012091978607673857+0j) [X0 X1 X2 Y7] +
(-0.0016354065601825016+0j) [X0 X1 Y2 X3] +
(-0.00020377653620314725+0j) [X0 X1 Y2 Y3] +
(0.0012091978607673857+0j) [X0 X1 Y2 Z3 Z5 Z6 X7] +
(0.0016354065601825016+0j) [X0 X1 Z2 X3] +
(0.004889539651547371+0j) [X0 X1 Z2 Y3] +
(-0.0038464821371521434+0j) [X0 X1 Z2 Z3 Z5 Y6 X7] +
(-0.0026372842763847577+0j) [X0 X1 Z2 Z5 Y6 X7] +
(0.004031651676556206+0j) [X0 X1 X4 Y7] +
(-0.004031651676556206+0j) [X0 X1 Y4 Z6 X7] +
(-0.009767493327321967+0j) [X0 X1 Z4 Z5 Y6 X7] +
(-0.00573584165076576+0j) [X0 X1 Z4 Y6 X7] +
(-0.0031369422783377675+0j) [X0 X1 Z5 X6 X7] +
(0.026669133009642746+0j) [X0 X1 Z5 Y6 X7] +
(-0.002746861819556288+0j) [X0 X1 Z5 Z6 X7] +
(-0.03893002781929343+0j) [X0 X1 X6 Y7] +
(0.004421732135337686+0j) [X0 X1 Y7] +
(0.0016354065601825016+0j) [X0 Y1 X2 X3] +
(0.00034734865840737586+0j) [X0 Y1 X2 Y3] +
(-0.0057416029429629735+0j) [X0 Y1 Y2

#### Find new FCI energy

In [6]:
new_Molecular_H_MATRIX = Hamilt.Get_sparse_Qubit_Hamiltonian_matrix(reduced_RE_LABELLED_Qubit_Hamiltonian)

from scipy.sparse.linalg import eigs
eig_values, eig_vectors = eigs(new_Molecular_H_MATRIX)
new_FCI_Energy = min(eig_values)

print('new_FCI = ', new_FCI_Energy, 'VS old FCI:', Hamilt.molecule.fci_energy)

new_FCI =  (-7.827946663038097+4.285536916939348e-17j) VS old FCI: -7.784460280267071


#### simplify Ansatz

In [7]:
# automate:
# reduced_Sec_Quant_CC_ops_ia, reduced_Sec_Quant_CC_ops_ijab, reduced_theta_parameters_ia, reduced_theta_parameters_ijab =ansatz_obj.Remove_NOON_terms(
#     NOON=NOON_spins_combined,
#     occ_threshold= 1.999,
#     unocc_threshold=1e-4,
#     indices_to_remove_list_manual=None, 
#     single_cc_amplitudes=Hamilt.molecule.single_cc_amplitudes,
#     double_cc_amplitudes=Hamilt.molecule.double_cc_amplitudes,
#     singles_hamiltonian=Hamilt.singles_hamiltonian,
#     doubles_hamiltonian=Hamilt.doubles_hamiltonian,
#     tol_filter_small_terms=None)

# manual
reduced_Sec_Quant_CC_ops_ia, reduced_Sec_Quant_CC_ops_ijab, reduced_theta_parameters_ia, reduced_theta_parameters_ijab =ansatz_obj.Remove_NOON_terms(
    NOON=NOON_spins_combined,
    indices_to_remove_list_manual=[0,1,6,7])

In [8]:
ia_terms, ijab_terms, ia_theta, ijab_theta = ansatz_obj.Get_ia_and_ijab_terms()
print('REDUCTION')
print('ia_terms', len(ia_terms), 'TO', len(reduced_Sec_Quant_CC_ops_ia))
print('ijab_terms', len(ijab_terms), 'TO', len(reduced_Sec_Quant_CC_ops_ijab))

REDUCTION
ia_terms 16 TO 6
ijab_terms 42 TO 6


In [9]:
Qubit_Op_list_Second_Quant_CC_Ops_ia, Qubit_Op_list_Second_Quant_CC_Ops_ijab = ansatz_obj.UCCSD_single_trotter_step(reduced_Sec_Quant_CC_ops_ia,
                                                                                                                    reduced_Sec_Quant_CC_ops_ijab)

reduced_CC_ijab = ansatz_obj.Remove_indices_from_CC_qubit_operators(Qubit_Op_list_Second_Quant_CC_Ops_ijab, list_of_qubit_indices_to_remove)


# RELABEL qubits!
reduced_RE_LABELLED_CC_ijab = ansatz_obj.Re_label_CC_qubit_operators( qubit_re_label_dict, 
                                                                   reduced_CC_ijab)
reduced_RE_LABELLED_CC_ijab

[0.125j [X0 X1 X2 Y3] +
 0.125j [X0 X1 Y2 X3] +
 -0.125j [X0 Y1 X2 X3] +
 0.125j [X0 Y1 Y2 Y3] +
 -0.125j [Y0 X1 X2 X3] +
 0.125j [Y0 X1 Y2 Y3] +
 -0.125j [Y0 Y1 X2 Y3] +
 -0.125j [Y0 Y1 Y2 X3],
 0.125j [X0 X1 X2 Z3 Z4 Y5] +
 0.125j [X0 X1 Y2 Z3 Z4 X5] +
 -0.125j [X0 Y1 X2 Z3 Z4 X5] +
 0.125j [X0 Y1 Y2 Z3 Z4 Y5] +
 -0.125j [Y0 X1 X2 Z3 Z4 X5] +
 0.125j [Y0 X1 Y2 Z3 Z4 Y5] +
 -0.125j [Y0 Y1 X2 Z3 Z4 Y5] +
 -0.125j [Y0 Y1 Y2 Z3 Z4 X5],
 0.125j [X0 X1 X2 Z3 Z4 Z5 Z6 Y7] +
 0.125j [X0 X1 Y2 Z3 Z4 Z5 Z6 X7] +
 -0.125j [X0 Y1 X2 Z3 Z4 Z5 Z6 X7] +
 0.125j [X0 Y1 Y2 Z3 Z4 Z5 Z6 Y7] +
 -0.125j [Y0 X1 X2 Z3 Z4 Z5 Z6 X7] +
 0.125j [Y0 X1 Y2 Z3 Z4 Z5 Z6 Y7] +
 -0.125j [Y0 Y1 X2 Z3 Z4 Z5 Z6 Y7] +
 -0.125j [Y0 Y1 Y2 Z3 Z4 Z5 Z6 X7],
 0.125j [X0 X1 X4 Y5] +
 0.125j [X0 X1 Y4 X5] +
 -0.125j [X0 Y1 X4 X5] +
 0.125j [X0 Y1 Y4 Y5] +
 -0.125j [Y0 X1 X4 X5] +
 0.125j [Y0 X1 Y4 Y5] +
 -0.125j [Y0 Y1 X4 Y5] +
 -0.125j [Y0 Y1 Y4 X5],
 0.125j [X0 X1 X4 Z5 Z6 Y7] +
 0.125j [X0 X1 Y4 Z5 Z6 X7] +
 -0.125j [X0 Y1 

In [10]:
reduced_CC_ia = ansatz_obj.Remove_indices_from_CC_qubit_operators(Qubit_Op_list_Second_Quant_CC_Ops_ia, list_of_qubit_indices_to_remove)

reduced_RE_LABELLED_CC_ia = ansatz_obj.Re_label_CC_qubit_operators( qubit_re_label_dict, 
                                                                   reduced_CC_ia)
reduced_RE_LABELLED_CC_ia

[-0.5j [X0 Z1 Y2] +
 0.5j [Y0 Z1 X2],
 -0.5j [X0 Z1 Z2 Z3 Y4] +
 0.5j [Y0 Z1 Z2 Z3 X4],
 -0.5j [X0 Z1 Z2 Z3 Z4 Z5 Y6] +
 0.5j [Y0 Z1 Z2 Z3 Z4 Z5 X6],
 -0.5j [X1 Z2 Y3] +
 0.5j [Y1 Z2 X3],
 -0.5j [X1 Z2 Z3 Z4 Y5] +
 0.5j [Y1 Z2 Z3 Z4 X5],
 -0.5j [X1 Z2 Z3 Z4 Z5 Z6 Y7] +
 0.5j [Y1 Z2 Z3 Z4 Z5 Z6 X7]]

#### Get reduced HF input state

In [11]:
print('old input = ', ansatz_obj.Get_BK_HF_state_in_OCC_basis())
print('BUT following indices removed:', list_of_qubit_indices_to_remove)
ansatz_obj.New_BK_HF_state(list_of_qubit_indices_to_remove)

old input =  [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
BUT following indices removed: [0, 1, 6, 7]


array([1., 0., 0., 0., 0., 0., 0., 0.])

# Ansatz Q circuit

In [12]:
N_QUBITS = Hamilt.molecule.n_qubits - len(list_of_qubit_indices_to_remove)
# or len(ansatz_obj.New_BK_HF_state(list_of_qubit_indices_to_remove))

N_electrons = int(sum(ansatz_obj.New_BK_HF_state(list_of_qubit_indices_to_remove)))

full_ansatz_Q_Circ = Ansatz_Circuit(reduced_RE_LABELLED_CC_ia, reduced_RE_LABELLED_CC_ijab,
             N_QUBITS, N_electrons, manual_HF_state=ansatz_obj.New_BK_HF_state(list_of_qubit_indices_to_remove))

ansatz_cirq_circuit = full_ansatz_Q_Circ.Get_Full_HF_UCCSD_QC(reduced_theta_parameters_ia, 
                                                              reduced_theta_parameters_ijab,
                                                              transformation='BK')

print(ansatz_cirq_circuit.to_text_diagram(transpose=True)) 

TypeError: __init__() got an unexpected keyword argument 'manual_HF_state'

In [40]:
from functools import reduce
from openfermion import qubit_operator_sparse
from scipy.linalg import expm

class Ansatz_lin_alg():
    """

    Build the ansatz state through linear algebra rather than quantum circuits.

    Args:
        PauliWord_str_Second_Quant_CC_JW_OP_list (list): List of Fermionic Operators (openfermion.ops._fermion_operator.FermionOperator)
        n_electrons (int): Number of electrons
        n_qubits (int): Number of qubits

    Attributes:
        reference_ket ():
        UCCSD_ops_matrix_list ():

    """
    def __init__(self, n_qubits):
        self.qubit_state_dict = {0: np.array([[1], [0]]),
                                 1: np.array([[0], [1]])}
        
        self.n_qubits = n_qubits


    def Get_reference_ket(self, qubit_state_list):
        """
        Takes in list of qubit states... e.g. [1,0,0] and returns corresponding ket.
        Not qubit indexing starts from left!

        [1,0,0] gives:

       array( [[0],
               [1],
               [0],
               [0],
               [0],
               [0],
               [0],
               [0]])

        Args:
            qubit_state_list (list): list of orbital indices that are OCCUPIED

        returns:
            reference_ket (np.array): KET of corresponding  occ no basis state
        """
        state = np.asarray(qubit_state_list[::-1], dtype=int)
        reference_ket = reduce(np.kron, [self.qubit_state_dict[bit] for bit in state])
        return reference_ket

    def Get_ia_UCC_matrix_NO_TROT(self, theta_ia, UCCSD_ops_ia):
        """
        operator = exp [ 𝑡02(𝑎†2𝑎0−𝑎†0𝑎2)+𝑡13(𝑎†3𝑎1−𝑎†1𝑎3)+𝑡0123(𝑎†3𝑎†2𝑎1𝑎0−𝑎†0𝑎†1𝑎2𝑎) ]
        """
        generator = scipy.sparse.csc_matrix((2 ** (self.n_qubits), 2 ** (self.n_qubits)), dtype=complex)
        for index_ia, qubit_op_ia in enumerate(UCCSD_ops_ia):
            generator += qubit_operator_sparse(qubit_op_ia, n_qubits=self.n_qubits) * theta_ia[index_ia]
        UCC_ia_operator = expm(generator)
        return UCC_ia_operator

    def Get_ijab_UCC_matrix_NO_TROT(self, theta_ijab, UCCSD_ops_ijab):
        """
        operator = exp [ 𝑡02(𝑎†2𝑎0−𝑎†0𝑎2)+𝑡13(𝑎†3𝑎1−𝑎†1𝑎3)+𝑡0123(𝑎†3𝑎†2𝑎1𝑎0−𝑎†0𝑎†1𝑎2𝑎) ]
        """
        generator = scipy.sparse.csc_matrix((2 ** (self.n_qubits), 2 ** (self.n_qubits)), dtype=complex)

        for index_ijab, qubit_op_ijab in enumerate(UCCSD_ops_ijab):
            generator += qubit_operator_sparse(qubit_op_ijab, n_qubits=self.n_qubits) * theta_ijab[index_ijab]
        UCC_ijab_operator = expm(generator)
        return UCC_ijab_operator

    def Get_ia_UCC_matrix_WITH_trot_SINGLE_STEP(self, theta_ia, UCCSD_ops_ia):
        """
        operator = exp [ 𝑡02(𝑎†2𝑎0−𝑎†0𝑎2)+𝑡13(𝑎†3𝑎1−𝑎†1𝑎3)+𝑡0123(𝑎†3𝑎†2𝑎1𝑎0−𝑎†0𝑎†1𝑎2𝑎) ]
        NOW
        operator = exp [𝑡02(𝑎†2𝑎0−𝑎†0𝑎2)] × exp[ 𝑡13(𝑎†3𝑎1−𝑎†1𝑎3)] × exp[𝑡0123(𝑎†3𝑎†2𝑎1𝑎0−𝑎†0𝑎†1𝑎2𝑎3)]
        """
        UCC_ia_operator = np.eye(2**(self.n_qubits), dtype=complex)
        for index_ia, qubit_op_ia in enumerate(UCCSD_ops_ia):
            qubit_op_matrix = qubit_operator_sparse(qubit_op_ia, n_qubits=self.n_qubits)
            UCC_ia_operator *= expm(qubit_op_matrix* theta_ia[index_ia])
        return UCC_ia_operator

    def Get_ijab_UCC_matrix_WITH_trot_SINGLE_STEP(self, theta_ijab, UCCSD_ops_ijab):
        """
        operator = exp [ 𝑡02(𝑎†2𝑎0−𝑎†0𝑎2)+𝑡13(𝑎†3𝑎1−𝑎†1𝑎3)+𝑡0123(𝑎†3𝑎†2𝑎1𝑎0−𝑎†0𝑎†1𝑎2𝑎) ]
        NOW
        operator = exp [𝑡02(𝑎†2𝑎0−𝑎†0𝑎2)] × exp[ 𝑡13(𝑎†3𝑎1−𝑎†1𝑎3)] × exp[𝑡0123(𝑎†3𝑎†2𝑎1𝑎0−𝑎†0𝑎†1𝑎2𝑎3)]
        """
        UCC_ijab_operator = np.eye(2**(self.n_qubits), dtype=complex)
        for index_ijab, qubit_op_ijab in enumerate(UCCSD_ops_ijab):
            qubit_op_matrix = qubit_operator_sparse(qubit_op_ijab, n_qubits=self.n_qubits)
            UCC_ijab_operator *= expm(qubit_op_matrix* theta_ijab[index_ijab])
        return UCC_ijab_operator

    def Get_Qubit_Hamiltonian_matrix(self, Qubit_MolecularHamiltonian):
        return qubit_operator_sparse(Qubit_MolecularHamiltonian)

    def Calc_energy_of_state(self, state_ket, Qubit_MolecularHamiltonianMatrix):
        state_bra = state_ket.transpose().conj()
        energy = state_bra.dot(Qubit_MolecularHamiltonianMatrix.dot(state_ket))
        return energy

In [46]:
aa = Ansatz_lin_alg(Hamilt.molecule.n_qubits-len(list_of_qubit_indices_to_remove))
# UU_ia = aa.Get_ia_UCC_matrix_NO_TROT(reduced_theta_parameters_ia, reduced_RE_LABELLED_CC_ia)
UU_ia =aa.Get_ia_UCC_matrix_WITH_trot_SINGLE_STEP(reduced_theta_parameters_ia, reduced_RE_LABELLED_CC_ia)

In [47]:
# UU_ijab = aa.Get_ia_UCC_matrix_NO_TROT(reduced_theta_parameters_ijab, reduced_RE_LABELLED_CC_ijab)
UU_ijab = aa.Get_ia_UCC_matrix_WITH_trot_SINGLE_STEP(reduced_theta_parameters_ijab, reduced_RE_LABELLED_CC_ijab)

In [48]:
print(ansatz_obj.New_BK_HF_state(list_of_qubit_indices_to_remove))

HF_state = aa.Get_reference_ket(ansatz_obj.New_BK_HF_state(list_of_qubit_indices_to_remove))

[1. 0. 0. 0. 0. 0. 0. 0.]


In [49]:
H_matrix = aa.Get_Qubit_Hamiltonian_matrix(reduced_RE_LABELLED_Qubit_Hamiltonian)

In [50]:
ket1 = UU_ia.dot(HF_state)
ket2 = UU_ijab.dot(ket1)
aa.Calc_energy_of_state(ket2, H_matrix)

array([[-6.78890231+0.j]])